In [3]:
import torch.utils.data as data
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from dataset import CloudDataset
import segmentation_models_pytorch as smp
from catalyst.dl.runner import SupervisedRunner
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import torch
from torch.utils.data import DataLoader
from utils import rle_decode, get_training_augmentation, get_preprocessing, get_validation_augmentation
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback
from albumentations import pytorch as AT
from PIL import Image
from tqdm import tqdm
import cv2
from  torch.utils.tensorboard import SummaryWriter
import os
import argparse
import torchsummary

In [9]:
encoder = 'resnet50'
encoder_weight = 'imagenet'
path = '../input/understanding_cloud_organization'
logdir = './logs/segmentation'
train = pd.read_csv(f'{path}/train.csv')
sub = pd.read_csv(f'{path}/sample_submission.csv')
net = "Unet"
class_num = 4
batch_size = 12

In [8]:
######### should go to logging
n_train = len(os.listdir(f'{path}/train_images'))
n_test = len(os.listdir(f'{path}/test_images'))
train['Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(
    lambda x: x.split('_')[0]).value_counts().value_counts()

train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])

sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])

id_mask_count = train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(
    lambda x: x.split('_')[0]).value_counts(). \
    reset_index().rename(columns={'index': 'img_id', 'Image_Label': 'count'})
train_ids, valid_ids = train_test_split(id_mask_count['img_id'].values, random_state=42,
                                        stratify=id_mask_count['count'], test_size=0.1)
test_ids = sub['Image_Label'].apply(lambda x: x.split('_')[0]).drop_duplicates().values

In [8]:
######### model parameter
ACTIVATION = torch.nn.Sigmoid
model = getattr(smp, net)(
    encoder_name=encoder,
    encoder_weights=encoder_weight,
    classes=class_num,
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(encoder, encoder_weight)
model.cuda()
# torchsummary.summary(model, (3,320, 640))

In [1]:
print("loading model")
model.load_state_dict(torch.load("../checkpoint/"+net+"/18"))
num_workers = 8
valid_dataset = CloudDataset(df=train, datatype='valid', img_ids=valid_ids,
                             transforms=get_validation_augmentation(),
                             preprocessing=get_preprocessing(preprocessing_fn), path=path)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
loaders = {
    "valid": valid_loader
}

loading model


NameError: name 'model' is not defined

In [2]:
print("validating")
validation_loss = []
for img, label in tqdm(loaders["valid"]):
    img = img.cuda()
    label = label.cuda()
    logit = model.predict(img)
    loss = criterion(logit, label)
    validation_loss.append(loss.item())

print("Validation loss: ", np.mean(validation_loss))
writer.add_scalar("validation_loss", np.mean(validation_loss), epoch)

validating


NameError: name 'tqdm' is not defined